In [9]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
import matplotlib.pylab as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import seaborn as sbn
import warnings
#import xgboost as xgb
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')
from sklearn.decomposition import PCA
import statsmodels.api as sm

## Part1: Final Prediction

In [10]:
lis_df = ['20181214_RFOutput.csv','20181214_HaomingOutput.csv']
df_main = pd.DataFrame()
a = 0
for i in lis_df:
    df = pd.read_csv(i)
    df = df[['Pred0','Pred1','Pred2','Pred3','Score']]
    df.columns=[str(a)+'Pred0',str(a)+'Pred1',str(a)+'Pred2',str(a)+'Pred3',str(a)+'Score']
    df_main = pd.concat([df_main, df], axis = 1)
    a = a+1

df_main.head()

,0Pred0,0Pred1,0Pred2,0Pred3,0Score,1Pred0,1Pred1,1Pred2,1Pred3,1Score
0,-0.368784,0.013594,-0.137000,-0.184137,0.049564,-0.139636,0.088890,-0.097058,-0.039421,0.049564
1,0.079639,-0.015930,0.056531,0.034818,0.154406,0.124321,0.134877,0.223301,0.209111,0.154406
2,0.066964,0.049295,-0.014329,0.160270,0.551646,0.263958,0.285271,0.089648,0.304978,0.551646
3,0.023889,0.055881,0.026003,0.144494,-0.280808,-0.007108,-0.033694,0.006291,0.184729,-0.280808
4,0.035926,0.055881,0.051805,0.144494,0.218957,-0.002909,-0.033694,0.011023,0.184729,0.218957


In [11]:
predictors = [x for x in df_main.columns if 'Pred' in x]
output = '0Score'
scaler = StandardScaler()

In [12]:
parameterRange = range(1,len(predictors))

IS_Overall = []
OS_Overall = []

for n in parameterRange :
    IS=[]
    OS=[]
    for i in range(1,20):
        # Don't cahnge anything here
        X_train, X_test, y_train, y_test = train_test_split(df_main[predictors], df_main[output], test_size=0.33, random_state=i*10)
        
        # Don't change anything here
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
        # Put PCA
        pca = PCA(n_components=n)
        pca.fit(X_train)
        X_train = pca.transform(X_train)
        X_test = pca.transform(X_test)
        
        model = sm.OLS(y_train, X_train).fit()
        ypredTr = model.predict(X_train)
        ypredTs = model.predict(X_test)
        
        # Don't change anything here
        IS.append(r2_score(y_train, ypredTr))
        OS.append(r2_score(y_test, ypredTs))
    
    # Don't change anything here
    print("Parameter: ", n, "IS :", np.mean(IS))
    print("Parameter: ", n, "OS :", np.mean(OS))
    IS_Overall.append(np.mean(IS))
    OS_Overall.append(np.mean(OS))

# Don't change anything here 
print(" Best Paramter:", parameterRange[np.argmax(OS_Overall)]," IS R2:",IS_Overall[np.argmax(OS_Overall)]," OS R2:", OS_Overall[np.argmax(OS_Overall)])    


Parameter:  1 IS : 0.570496228008
Parameter:  1 OS : 0.573301708251
Parameter:  2 IS : 0.572907154171
Parameter:  2 OS : 0.571873584689
Parameter:  3 IS : 0.573910162987
Parameter:  3 OS : 0.571198858081
Parameter:  4 IS : 0.575051183116
Parameter:  4 OS : 0.568537529706
Parameter:  5 IS : 0.637848229088
Parameter:  5 OS : 0.638121050311
Parameter:  6 IS : 0.638118023808
Parameter:  6 OS : 0.638340397241
Parameter:  7 IS : 0.638200537602
Parameter:  7 OS : 0.637882877887
 Best Paramter: 6  IS R2: 0.638118023808  OS R2: 0.638340397241


In [17]:
df_mainDup = df_main.copy(deep=True)
#df_features = pd.DataFrame(columns= predictors)
for i in range(4):
    np.random.seed(2016)
    Shuffle = list(np.random.permutation(len(df_main)))
    del Shuffle[int(i*len(df_main)/4):int((i+1)*(len(df_main))/4)]
    X_train = df_main.loc[Shuffle, predictors]
    y_train = df_main.loc[Shuffle, output]
    X_test = df_main.loc[:, predictors]
    y_test = df_main.loc[:, output]
    
    # Don't change anything here
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
        
    # Put PCA
    pca = PCA(n_components=7)
    pca.fit(X_train)
    X_train = pca.transform(X_train)
    X_test = pca.transform(X_test)
        
    model = sm.OLS(y_train, X_train).fit()
    ypredTr = model.predict(X_train)    
    
    ypredTs = model.predict(X_test)
    df_mainDup['Pred'+str(i)] = ypredTs
    print("Total R2", r2_score(y_test, ypredTs))


df_mainDup['FinalScore']=df_mainDup[['Pred0','Pred1','Pred2','Pred3']].mean(axis=1)
print("Final R2", r2_score(df_mainDup[output], df_mainDup['FinalScore']))

#df_mainDup.to_csv("20181214_FinalOutput.csv")

Total R2 0.516271659471
Total R2 0.48140983586
Total R2 0.432248337763
Total R2 0.492778980723
Final R2 0.631254485924


In [16]:
df_mainDup

NameError: name 'df_mainDup' is not defined

## Part 2: 

In [31]:
# import fine turnstile data
df_turnstile = pd.read_csv("20181207_FinalMainAndTurn.csv", parse_dates=['Date'], infer_datetime_format=True)
df_turnstile = df_turnstile[df_turnstile['Date']>=pd.datetime(2017,11,1)] 
print(df_turnstile.columns)
df_turnstile.head(2)

Index(['Unnamed: 0', 'index', 'Date', 'Station', 'Weekday', 'Maintenance',
       'EntryGain', 'Entries', 'Mean', 'Std', 'Score', 'Std1', 'F', 'N', 'R'],
      dtype='object')


,Unnamed: 0,index,Date,Station,Weekday,Maintenance,EntryGain,Entries,Mean,Std,Score,Std1,F,N,R
61,61,62,2017-11-04,G08,5,0,0,18066.0,17294.111111,2100.381837,0.220576,3499.426694,1.0,0.0,1.0
62,62,63,2017-11-11,G08,5,0,0,15894.0,16682.222222,1184.460297,-0.225243,3499.426694,1.0,0.0,1.0


In [28]:
# Import final maintenance data
df_maintenance = pd.read_csv("20181214_FinalMaintenance.csv", parse_dates=['Date'], infer_datetime_format=True)

# Calculate loss
df_maintenance['FinalScore'] = df_mainDup['FinalScore']
df_maintenance['Loss'] = (df_maintenance['Score'] * df_maintenance['Std1'])
df_maintenance = df_maintenance[df_maintenance['Loss']<df_maintenance['Loss'].quantile(0.95)]

# View data
print(df_maintenance.columns)

# View data
print(df_maintenance.loc[0,:])

Index(['Unnamed: 0', 'index', 'Date', 'Station', 'Weekday', 'Maintenance',
       'EntryGain', 'Entries', 'Mean', 'Std', 'Score', 'Std1', 'F', 'N', 'R',
       'Track', 'StationImp', 'Signal', 'Structural', 'Electrical',
       'ConnectionDay', 'Broken', 'Shuttle', 'OriginalConnection', 'NewConn',
       'NewConnRatio', 'LiveConn', 'Replacement', 'Saturday', 'StdRatio',
       'FinalScore', 'Loss'],
      dtype='object')
Unnamed: 0                              0
index                                  72
Date                  2018-01-13 00:00:00
Station                               G08
Weekday                                 5
Maintenance                             1
EntryGain                               1
Entries                             15721
Mean                              15547.6
Std                               1404.74
Score                           0.0495637
Std1                              3499.43
F                                    True
N                            

In [36]:
df_table1= pd.DataFrame()

for i in ['F','N','R']:
    df_turnTemp = pd.DataFrame()
    df_turnTemp.loc[0,'Line'] = i
    df_turnTemp.loc[0,'Entries'] = df_turnstile.loc[(df_turnstile[i]>0),'Entries'].sum()
    df_turnTemp.loc[0,'Loss'] = df_maintenance.loc[(df_maintenance[i]==True) & (df_maintenance['Loss']<0),'Loss'].sum()
    df_turnTemp.loc[0,'Gain'] = df_maintenance.loc[(df_maintenance[i]==True) & (df_maintenance['Loss']>0),'Loss'].sum()
    df_table1 = df_table1.append(df_turnTemp)

df_table1['%Loss'] = (df_table1['Loss']/df_table1['Entries'])*100
df_table1['Final Loss'] = df_table1['Loss'] + df_table1['Gain']
df_table1['%Gain'] = (-1.0 * df_table1['Gain']/df_table1['Loss'])*100
df_table1['%Final Loss'] = (df_table1['Final Loss']/df_table1['Entries'])*100

df_table1.to_csv("Results1.csv")

In [43]:
df_table2= pd.DataFrame()

for i in ['Track', 'StationImp', 'Signal', 'Structural', 'Electrical']:
    df_turnTemp = pd.DataFrame()
    df_turnTemp.loc[0,'Type'] = i
    df_turnTemp.loc[0,'Loss'] = df_maintenance.loc[(df_maintenance[i]==True),'Loss'].sum()
    df_table2 = df_table2.append(df_turnTemp)

df_table2['% of Total Loss'] = df_table2['Loss']*100/df_table2['Loss'].sum()
df_table2.to_csv('Results2.csv')

In [52]:
df_table3= pd.DataFrame()

for i in ['Track', 'StationImp', 'Signal', 'Structural', 'Electrical']:
    for j in ['F', 'N', 'R']:
        df_turnTemp = pd.DataFrame()
        df_turnTemp.loc[0,'Line'] = j
        df_turnTemp.loc[0,'Type'] = i
        df_turnTemp.loc[0,'Loss'] = df_maintenance.loc[(df_maintenance[i]==True) & (df_maintenance[j]==True),'Loss'].sum()
        df_table3 = df_table3.append(df_turnTemp)
        
df_table3['Loss'] = np.abs(df_table3['Loss'])
df_table3['% of Line'] = df_table3.apply(lambda x: 100* x['Loss']/df_table3.loc[df_table3['Line']==x['Line'],'Loss'].sum(), axis=1)
df_table3['% of Type'] = df_table3.apply(lambda x: 100* x['Loss']/df_table3.loc[df_table3['Type']==x['Type'],'Loss'].sum(), axis=1)

df_table3.to_csv('Results3.csv')

In [53]:
df_table3

,Line,Type,Loss,% of Line,% of Type
0,F,Track,50677.222222,7.171657,9.084642
0,N,Track,310535.678571,63.479892,55.668115
0,R,Track,196621.111111,64.386349,35.247243
0,F,StationImp,809.555556,0.114565,1.040257
0,N,StationImp,42581.507937,8.704538,54.716108
0,R,StationImp,34431.555556,11.275097,44.243635
0,F,Signal,577980.152381,81.793660,83.046677
0,N,Signal,51575.174603,10.543028,7.410543
0,R,Signal,66414.904762,21.748495,9.542780
0,F,Structural,29492.263889,4.173638,40.620132


In [21]:
# Extract month
df_maintenance['Month'] = df_maintenance['Date'].apply(lambda x: str(x.year)+"-"+str(x.month))
df_turnstile['Month'] = df_turnstile['Date'].apply(lambda x: str(x.year)+"-"+str(x.month))


In [22]:
df_lossMonth = pd.DataFrame(df_maintenance.groupby(['Month'])['Loss'].sum())
df_ridershipMonth = pd.DataFrame(df_turnstile.groupby(['Month'])['Entries'].sum())
df_month = pd.concat([df_lossMonth,df_ridershipMonth], axis=1)
df_month['Month']=df_month.index
df_month.drop(['2018-10'], inplace=True)
df_month.to_csv('20181214_LossMonth.csv')

In [23]:
df_maintenance.columns

Index(['Unnamed: 0', 'index', 'Date', 'Station', 'Weekday', 'Maintenance',
       'EntryGain', 'Entries', 'Mean', 'Std', 'Score', 'Std1', 'F', 'N', 'R',
       'Track', 'StationImp', 'Signal', 'Structural', 'Electrical',
       'ConnectionDay', 'Broken', 'Shuttle', 'OriginalConnection', 'NewConn',
       'NewConnRatio', 'LiveConn', 'Replacement', 'Saturday', 'StdRatio',
       'FinalScore', 'Loss', 'Month'],
      dtype='object')

In [ ]:
df_monthType =pd.DataFrame()

for i in ['Track', 'StationImp', 'Signal', 'Structural', 'Electrical']:
    df_lossMonth = pd.DataFrame(df_maintenance.groupby(['Month',i], as_index=False)['Loss'].sum())
    df_lossMonth['Type'] = i
    df_lossMonth = df_lossMonth.loc[(df_lossMonth['Month']!='2018-10')&(df_lossMonth[i]==True),['Month','Loss','Type']]
    df_monthType = df_monthType.append(df_lossMonth)
    
df_monthType['Loss']=-1.0*abs(df_monthType['Loss'])
df_monthType.replace(to_replace = np.nan, value=0, inplace= True)
df_monthType.to_csv('20181214_monthType.csv')

In [ ]:
df_Line =pd.DataFrame()

for i in ['F','N','R']:
    df_lossMonth = pd.DataFrame(df_maintenance.groupby(['Month',i], as_index=False)['Loss'].sum())
    df_ridershipMonth = pd.DataFrame(df_turnstile.groupby(['Month',i], as_index=False)['Entries'].sum())
    df_month1 = df_lossMonth.merge(df_ridershipMonth, on=['Month',i], how='outer')
    df_month1['Line'] = i
    df_month1 = df_month1.loc[(df_month1['Month']!='2018-10')&(df_month1[i]==True),['Month','Loss','Entries','Line']]
    df_monthLine = df_monthLine.append(df_month1)
    
df_monthLine['Loss']=-1.0*abs(df_monthLine['Loss'])
df_monthLine.replace(to_replace = np.nan, value=0, inplace= True)
df_monthLine.to_csv('20181214_monthLine.csv')